use newpycaret env


In [5]:
# packages
import snowflake.connector
import pandas as pd
import os
import numpy as np
from autots import AutoTS
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
from tqdm import tqdm
from datetime import datetime, timedelta
from dateutil import relativedelta

2. Load Data

---

### To Do: copy and paste in to a new chunk, enter credentials and run to save in environment. Then delete chunk

%env snowflakeuser=<your_snowflake_username> <br>
%env snowflakepass=<your_snowflake_password>

In [6]:
# Query Snowflake

# Snowflake connection parameters
connection_params = {
    "user": os.environ['snowflakeuser'],
    "password": os.environ['snowflakepass'],
    "account": "zib52348.us-east-1",
    "role": "ACCOUNTADMIN",
    "warehouse": "REPORTING",
    "database": "ANALYTICS",
    "schema": "FORECASTING",
}

# Establish a connection to Snowflake
conn = snowflake.connector.connect(**connection_params)

with open('net_sales_query.sql', 'r') as query:
    # connection == the connection to your database, in your case prob_db
    df_initial = pd.read_sql_query(query.read(), conn)

# Close the connection
conn.close()

C:\Users\crudek\AppData\Local\Temp\ipykernel_24392\2623517584.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_initial = pd.read_sql_query(query.read(), conn)


In [10]:
##### INITIAL VARIABLE DECLARATION #####

# Get the current date
current_date = datetime.today()

# Calculate the first day of the current month
first_day_of_current_month = pd.to_datetime(current_date.replace(day=1))

# Calculate the first day of the next month
first_day_of_next_month = datetime(current_date.year, current_date.month + 1, 1)

# Calculate the first day of the last month
last_day_of_last_month = first_day_of_current_month - timedelta(days=1)
first_day_of_last_month = last_day_of_last_month.replace(day=1)




### MANUAL INPUTS ###

# how much data do you want to keep to train models. 
#end_of_data = first_day_of_last_month # the default which is the max date this should ever be because it would include all months that have full data
end_of_data = pd.to_datetime('2022-12-01')

# forecast horizon = how many months in to the future you want to forecast. So, we will forecast this many months past the above end_of_data
fh = 12




### FORECAST HORIZON (FH) DATAFRAME ###

end_of_data_next_month = end_of_data + relativedelta.relativedelta(months=1, day=1)
end_of_data_df = pd.DataFrame({'end_of_data': [end_of_data_next_month]})
end_of_data_df['end_of_data'] = pd.to_datetime(end_of_data_df['end_of_data'])

# Create a date range for the next 12 months
next_12_months = pd.date_range(start=end_of_data_df['end_of_data'].iloc[0], periods=fh, freq='MS')

fh_dates_df = pd.DataFrame({'MONTH': next_12_months})




### INITIAL DATA PREPARATION ###

# create copy of df_d
df = df_initial.copy(deep=True)

# convert month field to date
df["MONTH"] = pd.to_datetime(df["MONTH"])

# Some random months will have data that we want to remove (* Want to test without July though)
df = df[["DEP_ENT", "MONTH", "NET_SALES"]]  # select fields of interest
df = df.sort_values(['DEP_ENT', 'MONTH'])  # reorder dataframe

# remove data after the 'end of data' setting above
df_sub = df[df['MONTH'] <= end_of_data]

# create series
df_s = df_sub.set_index(['DEP_ENT','MONTH'])['NET_SALES']
# convert back to dataframe
df_d = df_s.to_frame()
#reset index
df_d.reset_index(inplace=True)


# list of each dep-ent
all_dep_ent = df_d['DEP_ENT'].unique()


### IMPUTE MISSIG VALUES WITH MONTH AND MEAN ###

# Create empty dataframe
df_subset_all = pd.DataFrame()

for i in tqdm(all_dep_ent):
    
    temp_subset = df_d[df_d['DEP_ENT'] == i]
    
    # Define the minimum and maximum dates
    min_date = min(temp_subset['MONTH'])
    max_date = max(temp_subset['MONTH'])

    # Generate a list of dates for each month in between
    date_range = []
    current_month = min_date.replace(day=1)
    while current_month <= max_date:
        date_range.append(current_month)
        current_month = current_month + timedelta(days=32)
        current_month = current_month.replace(day=1)

    # Create a DataFrame from the list of dates
    date_range_df = pd.DataFrame({'MONTH': date_range})

    df_merged = pd.merge(date_range_df, temp_subset, on='MONTH', how='left')

    #Finding the mean of the column having NaN
    mean_value = df_merged['NET_SALES'].mean()
    
    # Replace NaNs in column S2 with the
    # mean of values in the same column
    df_merged['NET_SALES'].fillna(value=mean_value, inplace=True)
    
    df_merged['DEP_ENT'] = i

    df_subset_all = df_subset_all.append(df_merged)




### FILL DATES THROUGH END OF FORECAST HORIZON ###

# Create empty dataframe
df_subset_all2 = pd.DataFrame()

for i in tqdm(all_dep_ent):
    # filter to one dep_ent
    temp_subset2 = df_subset_all[df_subset_all['DEP_ENT'] == i]
    # temp result
    temp_result2 = pd.merge(temp_subset2, fh_dates_df, on='MONTH', how='outer')
    temp_result2['DEP_ENT'] = i
    
    # combine for all dep_ents
    df_subset_all2 = df_subset_all2.append(temp_result2)





### REGRESSOR DATA PREPARATION ###

# create copy of df_d
df_d_regress = df_d.copy(deep=True)

# Prepare regressors

# Define the prime COVID period
cov_start_date = pd.Timestamp(2020, 1, 1)
cov_end_date = pd.Timestamp(2021, 12, 1)

# Create the binary dummy variable
df_d_regress['COVID'] = df_d_regress['MONTH'].apply(lambda date: '1' if cov_start_date <= date <= cov_end_date else '0')

# Amazon prime day shipments

# Define the dictionary
amzn_ship_dict = {
    'MONTH': [pd.Timestamp(2016, 7, 1), pd.Timestamp(2017, 5, 1), pd.Timestamp(2018, 6, 1), pd.Timestamp(2019, 6, 1), pd.Timestamp(2020, 6, 1),
              pd.Timestamp(2020, 10, 1), pd.Timestamp(2021, 5, 1), pd.Timestamp(2022, 5, 1), pd.Timestamp(2023, 5, 1), pd.Timestamp(2023, 6, 1), pd.Timestamp(2023, 7, 1)],
    'AMZN': ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
}

# Create a DataFrame from the dictionary
amzn_ship_df = pd.DataFrame(amzn_ship_dict)
# add Amazon department
amzn_ship_df['DEP_ENT'] = '250_155'

# combine in to single table
df_d_regress1 = df_d_regress.merge(
    amzn_ship_df, on=['DEP_ENT', 'MONTH'], how='left')

df_d_regress1['AMZN'] = df_d_regress1['AMZN'].fillna(0)

  0%|          | 0/14 [00:00<?, ?it/s]

C:\Users\crudek\AppData\Local\Temp\ipykernel_24392\927946470.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_subset_all = df_subset_all.append(df_merged)
C:\Users\crudek\AppData\Local\Temp\ipykernel_24392\927946470.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_subset_all = df_subset_all.append(df_merged)
C:\Users\crudek\AppData\Local\Temp\ipykernel_24392\927946470.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_subset_all = df_subset_all.append(df_merged)
C:\Users\crudek\AppData\Local\Temp\ipykernel_24392\927946470.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_subset_all = df_subset_all.append(df_

In [13]:
df_subset_all

,MONTH,DEP_ENT,NET_SALES
0,2016-01-01,160_155,1076855.07
1,2016-02-01,160_155,1305615.57
2,2016-03-01,160_155,1774890.87
3,2016-04-01,160_155,852505.23
4,2016-05-01,160_155,3399714.50
...,...,...,...
42,2022-08-01,260_155,3398.93
43,2022-09-01,260_155,5377.52
44,2022-10-01,260_155,2673.52
45,2022-11-01,260_155,2998.44


---


In [12]:
### Time Series Loop w/ Regressors ###

# Create empty dataframes
all_predictions = pd.DataFrame()

# list of each dep-ent
de_list = df_d_regress1['DEP_ENT'].unique()

for i in tqdm(de_list):

    df_subset = df_d_regress1[df_d_regress1['DEP_ENT'] == i]
    df_subset_trim = df_subset[df_subset['MONTH'] <= end_of_data]
    df_subset_trim = df_subset_trim[['MONTH', 'NET_SALES']]

    # split the train and original df

    # train
    df_subset_train = df_subset[df_subset['MONTH'] <= end_of_data]
    df_subset_train = df_subset_train[["MONTH", "COVID", "AMZN"]]

    # for future forecast
    df_subset_fcast = df_subset[df_subset['MONTH'] > end_of_data]
    df_subset_fcast = df_subset_fcast[["MONTH", "COVID", "AMZN"]]

    # model
    model = AutoTS(
        forecast_length=fh,
        frequency='infer',
        # prediction_interval=0.95,
        # ensemble='simple',
        # models_mode='deep',
        # model_list = 'univariate', # or could do a list like ['ARIMA','ETS']
        max_generations=3,
        num_validations=3,
        no_negatives=True,
        n_jobs='auto'
    )

    model = model.fit(
        df_subset_trim,
        future_regressor=df_subset_train,
        date_col='MONTH',
        value_col='NET_SALES',
    )

    # print(model)

    # create prediction
    prediction = model.predict(future_regressor=df_subset_fcast, forecast_length=fh)

    # temp fcast dataframe
    temp_fcasts = prediction.forecast

    # rename
    temp_fcasts.rename(columns={'index': 'MONTH'}, inplace=True)

    temp_fcasts['DEP_ENT'] = i  # add dep

    # append to master dataframe
    all_predictions = all_predictions.append(temp_fcasts)

  0%|          | 0/14 [00:00<?, ?it/s]

Using 6 cpus for n_jobs.
Data frequency is: MS, used frequency is: MS
Model Number: 1 with model ARIMA in generation 0 of 3
Model Number: 2 with model AverageValueNaive in generation 0 of 3
Model Number: 3 with model AverageValueNaive in generation 0 of 3
Model Number: 4 with model AverageValueNaive in generation 0 of 3
Model Number: 5 with model DatepartRegression in generation 0 of 3
Model Number: 6 with model DatepartRegression in generation 0 of 3
Model Number: 7 with model DatepartRegression in generation 0 of 3


c:\Users\crudek\AppData\Local\anaconda3\envs\newpycaret\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Model Number: 8 with model DatepartRegression in generation 0 of 3
Template Eval Error: ImportError('Tensorflow not available, install with pip install tensorflow.') in model 8 in generation 0: DatepartRegression
Model Number: 9 with model ETS in generation 0 of 3
Model Number: 10 with model ETS in generation 0 of 3
Model Number: 11 with model GLM in generation 0 of 3
Model Number: 12 with model GLM in generation 0 of 3


c:\Users\crudek\AppData\Local\anaconda3\envs\newpycaret\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


...

32 - ARIMA with avg smape 79.86: 
Model Number: 33 of 84 with model ARDL for Validation 3
33 - ARDL with avg smape 50.25: 
Model Number: 34 of 84 with model ARDL for Validation 3
34 - ARDL with avg smape 50.25: 
Model Number: 35 of 84 with model SeasonalNaive for Validation 3
Template Eval Error: Exception('Transformer RegressionFilter failed on fit') in model 35 in generation 0: SeasonalNaive
Model Number: 36 of 84 with model ARIMA for Validation 3
36 - ARIMA with avg smape 96.86: 
Model Number: 37 of 84 with model LastValueNaive for Validation 3
Template Eval Error: Exception('Transformer RegressionFilter failed on fit') in model 37 in generation 0: LastValueNaive
Model Number: 38 of 84 with model SectionalMotif for Validation 3
38 - SectionalMotif with avg smape 22.23: 
Model Number: 39 of 84 with model SectionalMotif for Validation 3
39 - SectionalMotif with avg smape 24.12: 
Model Number: 40 of 84 with model ARDL for Validation 3
40 - ARDL with avg smape 63.34: 
Model Number: 41 o

c:\Users\crudek\AppData\Local\anaconda3\envs\newpycaret\lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=2.9706e-24): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


71 - DatepartRegression with avg smape 24.44: 
Model Number: 72 of 84 with model ConstantNaive for Validation 3
72 - ConstantNaive with avg smape 50.87: 
Model Number: 73 of 84 with model UnivariateMotif for Validation 3
73 - UnivariateMotif with avg smape 88.24: 
Model Number: 74 of 84 with model WindowRegression for Validation 3
74 - WindowRegression with avg smape 85.5: 
Model Number: 75 of 84 with model WindowRegression for Validation 3
75 - WindowRegression with avg smape 64.59: 
Model Number: 76 of 84 with model WindowRegression for Validation 3
76 - WindowRegression with avg smape 72.12: 
Model Number: 77 of 84 with model MetricMotif for Validation 3
Template Eval Error: ValueError('window shape cannot be larger than input array shape') in model 77 in generation 0: MetricMotif
Model Number: 78 of 84 with model UnivariateMotif for Validation 3
78 - UnivariateMotif with avg smape 69.37: 
Model Number: 79 of 84 with model MetricMotif for Validation 3
79 - MetricMotif with avg smape

 21%|██▏       | 3/14 [08:08<29:50, 162.74s/it]


ValueError: future_regressor and X index failed to align

In [ ]:
### AFTER THE LOOP ###

# rename index col
all_predictions.rename(columns={'index': 'MONTH', 'NET_SALES': 'PRED'}, inplace=True)

# initial sales pull
sales = df_initial
# select cols
sales = sales[["DEP_ENT", "MONTH", "NET_SALES"]]
# only include sales data for the full months we have
first_of_month = datetime.today().replace(day=1).date()
sales = sales[sales['MONTH'] < pd.to_datetime(first_of_month)]

# combine prediction data and original sales data
merged = pd.merge(all_predictions, sales, how='left', on=['DEP_ENT', 'MONTH'])

In [18]:
### Budget ###

# Establish a connection to Snowflake
conn = snowflake.connector.connect(**connection_params)

with open('budget_query.sql', 'r') as query:
    # connection == the connection to your database, in your case prob_db
    budg = pd.read_sql_query(query.read(), conn)

# Close the connection
conn.close()

# convert month to datetime
budg["MONTH"] = pd.to_datetime(budg["MONTH"])
# select cols
budg = budg[["MONTH", "BUDGET_AMOUNT", "DEP_ENT"]]


WARNING - (py.warnings._showwarnmsg) - C:\Users\crudek\AppData\Local\Temp\ipykernel_12204\3534014428.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  budg = pd.read_sql_query(query.read(),conn)



In [19]:
# combine prediction/sales data with budget data
merged2 = pd.merge(merged, budg, how='left', on=['DEP_ENT', 'MONTH'])


In [20]:
# write csv
merged2.to_csv('auto_ts_results.csv')
